In [35]:
import pandas as pd

# Load the Excel file
file_path = 'C:/1作品/clothse_excel/train1800.xlsx'
data = pd.read_excel(file_path)

# Display the first few rows of the dataframe
data.head()


,Unnamed: 0,date,wether,max_tem,min_tem,mean_tem,average_humidity,average_wind_speed(m/s),sensible_temperature,name,label1,label2,label3,season
0,0,2021-01-09,曇,6.8,3.9,5.35,31,2.3,0.733549,p7l7560000052ulh.jpg,coat,V-neck top,suit pants,Winter
1,1,2021-01-09,曇,6.8,3.9,5.35,31,2.3,0.733549,p7l7560000052ulv.jpg,coat,sweater,pants,Winter
2,2,2021-01-09,曇,6.8,3.9,5.35,31,2.3,0.733549,p7l7560000052um2.jpg,coat,suit jackets,jeans,Winter
3,3,2021-01-09,曇,6.8,3.9,5.35,31,2.3,0.733549,p7l7560000052um9.jpg,coat,sweater,pants,Winter
4,4,2021-01-09,曇,6.8,3.9,5.35,31,2.3,0.733549,p7l7560000052umg.jpg,short down jacket,turtleneck sweater,pants,Winter


In [36]:
# 使用OneHotEncoder进行独热编码
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
import numpy as np

# 对分类特征进行独热编码
onehot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
wether_encoded = onehot_encoder.fit_transform(data[['wether']])
season_encoded = onehot_encoder.fit_transform(data[['season']])

# 合并所有特征
features = np.hstack([
    data[['max_tem', 'min_tem', 'mean_tem', 'average_humidity', 'average_wind_speed(m/s)', 'sensible_temperature']],
    wether_encoded,
    season_encoded
])


c:\Users\koyama\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\koyama\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [37]:
# 将每行的标签转换为一个列表
labels_list = data[['label1', 'label2', 'label3']].values.tolist()

# 确保所有标签都转换为字符串类型
labels_list_str = [[str(label) for label in sublist] for sublist in labels_list]

# 使用MultiLabelBinarizer处理转换后的标签列表
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels_list_str)

# 检查处理后的标签形状
print("Labels shape:", labels.shape)



Labels shape: (1799, 33)


In [38]:
onehot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
wether_encoded = onehot_encoder.fit_transform(data[['wether']])
season_encoded = onehot_encoder.fit_transform(data[['season']])

features = np.hstack([
    data[['max_tem', 'min_tem', 'mean_tem', 'average_humidity', 'average_wind_speed(m/s)', 'sensible_temperature']].values,
    wether_encoded,
    season_encoded
])

assert not np.isnan(features).any(), "Features contain NaN values"


c:\Users\koyama\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\koyama\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [39]:
from sklearn.model_selection import train_test_split

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [40]:
from sklearn.ensemble import RandomForestClassifier

# 初始化随机森林分类器
rf_classifier = RandomForestClassifier(random_state=42)

# 训练模型
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [41]:
# 使用模型进行预测
y_pred = rf_classifier.predict(X_test)


In [42]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.019444444444444445

In [43]:
from sklearn.metrics import classification_report

# 评估模型性能
print(classification_report(y_test, y_pred, zero_division=0))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        42
           1       0.88      0.64      0.74       165
           2       0.00      0.00      0.00         0
           3       0.15      0.08      0.11        25
           4       0.00      0.00      0.00        50
           5       0.67      0.18      0.29        11
           6       0.00      0.00      0.00        15
           7       0.61      0.26      0.37        42
           8       0.53      0.15      0.24       118
           9       0.00      0.00      0.00        12
          10       0.00      0.00      0.00        11
          11       0.00      0.00      0.00         2
          12       0.00      0.00      0.00        10
          13       0.00      0.00      0.00        32
          14       0.00      0.00      0.00        22
          15       0.47      0.24      0.32       112
          16       0.72      0.23      0.34        93
          17       0.00    

In [44]:
from sklearn.metrics import multilabel_confusion_matrix

# 计算多标签混淆矩阵
mcm = multilabel_confusion_matrix(y_test, y_pred)

# 打印每个标签的混淆矩阵
for i, matrix in enumerate(mcm):
    print(f"Label {mlb.classes_[i]} 的混淆矩阵:")
    print(matrix, "\n")


Label Hoodie 的混淆矩阵:
[[318   0]
 [ 42   0]] 

Label Unknown 的混淆矩阵:
[[180  15]
 [ 59 106]] 

Label V-neck top 的混淆矩阵:
[[360   0]
 [  0   0]] 

Label base layer 的混淆矩阵:
[[324  11]
 [ 23   2]] 

Label coat 的混淆矩阵:
[[310   0]
 [ 50   0]] 

Label denim jacket 的混淆矩阵:
[[348   1]
 [  9   2]] 

Label dress 的混淆矩阵:
[[345   0]
 [ 15   0]] 

Label jackets 的混淆矩阵:
[[311   7]
 [ 31  11]] 

Label jeans 的混淆矩阵:
[[226  16]
 [100  18]] 

Label knitted coat 的混淆矩阵:
[[348   0]
 [ 12   0]] 

Label knitted pants 的混淆矩阵:
[[349   0]
 [ 11   0]] 

Label leather pants 的混淆矩阵:
[[358   0]
 [  2   0]] 

Label long down jacket 的混淆矩阵:
[[350   0]
 [ 10   0]] 

Label long skirt 的混淆矩阵:
[[328   0]
 [ 32   0]] 

Label long-sleeved dress 的混淆矩阵:
[[338   0]
 [ 22   0]] 

Label pants 的混淆矩阵:
[[218  30]
 [ 85  27]] 

Label shirts 的混淆矩阵:
[[259   8]
 [ 72  21]] 

Label short down jacket 的混淆矩阵:
[[338   0]
 [ 22   0]] 

Label short pants 的混淆矩阵:
[[345   0]
 [ 15   0]] 

Label short skirt 的混淆矩阵:
[[344   0]
 [ 16   0]] 

Label short trench coa

In [45]:
from sklearn.metrics import confusion_matrix

# 初始化计数器
true_positive = 0
false_positive = 0
true_negative = 0
false_negative = 0

# 遍历每个标签，累加混淆矩阵的值
for i in range(y_test.shape[1]):
    cm = confusion_matrix(y_test[:, i], y_pred[:, i])
    # 确保混淆矩阵是2x2的
    if cm.shape == (2, 2):
        true_negative += cm[0, 0]
        false_positive += cm[0, 1]
        false_negative += cm[1, 0]
        true_positive += cm[1, 1]
    else:
        # 处理那些在预测或实际标签中只有一类的情况
        # 这通常表示模型对于某些标签没有做出任何正类预测
        print(f"Label index {i} 的混淆矩阵不是2x2，可能因为没有预测到正类或测试集中不包含正类。")

# 展示整体混淆矩阵
print(f"整体混淆矩阵:")
print(f"真负例（TN）: {true_negative}")
print(f"假正例（FP）: {false_positive}")
print(f"假负例（FN）: {false_negative}")
print(f"真正例（TP）: {true_positive}")


Label index 2 的混淆矩阵不是2x2，可能因为没有预测到正类或测试集中不包含正类。
Label index 29 的混淆矩阵不是2x2，可能因为没有预测到正类或测试集中不包含正类。
整体混淆矩阵:
真负例（TN）: 9968
假正例（FP）: 129
假负例（FN）: 827
真正例（TP）: 236
